In [16]:
# ignore deprecation warnings ('safe' as long as we don't update packages)
from warnings import filterwarnings
filterwarnings("ignore")

from math import sqrt, log, inf
import copy

import gym
from gym_go.gogame import turn, random_weighted_action

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib
import numpy as np

# Constants

In [99]:
UCB_C = 30

# Board will be a BOARD_SIZE * BOARD_SIZE board (BOARD_SIZE**2)
BOARD_SIZE = 5

# Indication layer channel indices
BLACK = 0
WHITE = 1
TURN = 2
INVALID = 3
PASS = 4
DONE = 5

# Monte Carlo Tree Search

1. Selection
    - Taverse the tree to find greatest UCB-score
2. Expansion
    - If the selected leaf node has been visited before expand by adding weighted game action
3. Rollout
    - Simulate the game until end-condition from the expanded leaf
4. Back-propagation
    - Updating the value of each ancestor node of the expanded leaf


In [92]:
# TODO: GymGo.gogame.move/validmove/elns for å nulle ut actionspace
def get_legal_move(env):
    board_shape = env.state().shape[1:]
    pass_id = np.prod(board_shape)
    action = env.action_space.sample() # pick random action
    action2d = action // board_shape[0], action % board_shape[1], action
    while action2d[2] != pass_id and env.state()[INVALID, action2d[0], action2d[1]] == 1:
        action = env.action_space.sample() # pick random action
        action2d = action // board_shape[0], action % board_shape[1], action
    return action2d[2]

def equals(state_a, state_b):
    for i in range(0, 6):
        if not np.array_equal(state_a[i], state_b[i]):
            return False
    return True


In [93]:
class Node():
    def __init__(self, env, parent, action):
        self.env : gym.Env = env # This env will be altered by the other player
        self.value : int = 0 # Value estimate
        self.trials : int = 0 # Number of trials for this node
        self.parent : Node = parent # Parent node of this node
        self.children : list[Node] = [] # List of children of this node
        self.action : int = action # The step action made by this node
    
    # calculate a Upper Confidence Bound
    def ucb(self, total_trials):
        return self.value + ( UCB_C * sqrt(log(total_trials) / self.trials) )
    
    # Add a new node to a leaf node
    def expansion(self, move_weights):
        if self.env.done:
            return
        board_shape = self.env.state().shape[1:]
        pass_id = np.prod(board_shape)
        for action in range(0, pass_id + 1):
            action2d = action // board_shape[0], action % board_shape[1]
            if action == pass_id or self.env.state()[3, action2d[0], action2d[1]] == 0:
                child_env = copy.deepcopy(self.env)
                child_env.step(action)
                child = Node(child_env, self, action)
                child.value = move_weights[action]
                self.children.append(child)

    # Simulate game from current move until end-condition returning the score
    def rollout(self, move_selection_method):
        if self.env.done:
            return self.env.reward()
        
        rollout_env = copy.deepcopy(self.env)
        rollout_result = 0
        done = False
        while not done:
            random_action = move_selection_method(rollout_env)
            _, reward, done, _ = rollout_env.step(random_action)
            rollout_result += reward
        return rollout_result

In [94]:
class Monte_Carlo_Tree_Search():
    def __init__(self, size, ml_model):
        self.env : gym.Env = gym.make('gym_go:go-v0', size=size, reward_method='heuristic')
        self.env.reset()
        self.number_of_trials : int = 0
        self.root = Node(self.env, None, None)
        self.ml_model = ml_model
    
    def get_move_weights(self, state):
        # Do something to get this from ml_model
        move_weights = np.ones(BOARD_SIZE ** 2 + 1)
        
        board_shape = state.shape[1:]
        for i in range(len(move_weights) - 1):
            action2d = i // board_shape[0], i % board_shape[1], i
            if state[INVALID, action2d[0], action2d[1]] == 1:
                move_weights[i] = 0.0

        return move_weights

    def get_weighted_move(self, env : gym.Env):
        move_weights = self.get_move_weights(env.state())
        return random_weighted_action(move_weights)

    # Update scores of all parent nodes after rollout
    def back_propagation(self, rollout_node: Node, rollout_result):
        current_node = rollout_node
        while current_node != None:
            current_node.trials += 1
            if turn(self.env.state()) == BLACK:
                current_node.value -= rollout_result
            if turn(self.env.state()) == WHITE:                
                current_node.value += rollout_result
            current_node = current_node.parent
        self.number_of_trials += 1
    
    # find and return the leaf node with the highest UCB-score 
    def selection(self):
        selected_child = self.root
        current_node = self.root
        while len(current_node.children) > 0:
            selected_child = current_node.children[0]
            current_best_ucb = -inf

            for child in current_node.children:
                if child.trials == 0:
                    return child

                child_ucb = child.ucb(self.number_of_trials)

                if child_ucb > current_best_ucb:
                    selected_child = child
                    current_best_ucb = child_ucb

            current_node = selected_child

        return selected_child
        
    def run(self, iterations):
        selected_node = self.root
        selected_node.expansion(self.get_move_weights(selected_node.env.state()))
        selected_node = self.root.children[0]

        run = 0
        while run < iterations:
            selected_node = self.selection()
            # print("Run:", run, ": Selection", selected_node.action)

            if selected_node.env.done:
                self.back_propagation(selected_node, selected_node.env.reward())
                run += 1
                continue

            if selected_node.trials > 0:
                # print("Run:", run, ": Expansion:")
                selected_node.expansion(self.get_move_weights(selected_node.env.state()))
                selected_node = selected_node.children[0]

            rollout_result = selected_node.rollout(self.get_weighted_move)
            # print("Run:", run, ": Rollout", rollout_result)
            
            self.back_propagation(selected_node, rollout_result)
            run += 1
    
    def find_node_from_state(self, node: Node, state):
        if equals(node.env.state(), state):
            return node

        for child in node.children:
            if equals(child.env.state(), state):
                return child
            
            res = self.find_node_from_state(child, state)
            if res != None and equals(res.env.state(), state):
                return res

    def get_move_from_env(self, env):
        node = self.find_node_from_state(self.root, env.state())
        
        if node != None:
            print("Found node for state:")
            node.env.render()
            print("Finding best move from:", len(node.children), "available explored moves.")
            best_child = None
            current_best_value = -inf
            for child in node.children:
                if child.value > current_best_value:
                    best_child = child
                    current_best_value = child.value

            if best_child != None:
                return best_child
        
        return None

    def get_training_data_from_tree(self):
        training_data = []
        self.get_training_data_from_node(training_data, self.root)
        return training_data
        
    def get_training_data_from_node(self, training_data : list, current_node):
        node_tuple = [current_node.env.state(), [0] * (BOARD_SIZE ** 2 + 1)]
        for child in current_node.children:
            node_tuple[1][child.action] = child.value
            training_data.append(node_tuple)
            self.get_training_data_from_node(training_data, child)
        

In [95]:
model = Monte_Carlo_Tree_Search(5, None)
model.run(100)

In [ ]:
# Sjekker hvor utforska treet er for å teste UCB_C konstanten

def print_tried_children(current_node : Node):
    for child in current_node.children:
        if child.trials == 0:
            continue
        print("Child", child.action, ":")
        print("Number of children:", len(child.children))
        print("Value:", child.value, "Number of trials:", child.trials, "UCB:", child.ucb(model.number_of_trials))

current_node = model.root
while len(current_node.children) > 0:
    most_tried = current_node.children[0]
    for child in current_node.children:
        if child.trials == 0:
            continue
        print("Child", child.action, ":")
        child.env.render()
        print("Number of children:", len(child.children))
        print("Value:", child.value, "Number of trials:", child.trials, "UCB:", child.ucb(model.number_of_trials))
        print_tried_children(child)
        if child.trials > most_tried.trials:
            most_tried = child
        current_node = most_tried

Child 0 :
	0 1 2 3 4 
0	○═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Number of children: 25
Value: -267.0 Number of trials: 25 UCB: -254.1242038422639
Child 1 :
Number of children: 0
Value: 48.0 Number of trials: 1 UCB: 112.37898078868042
Child 2 :
Number of children: 0
Value: 58.0 Number of trials: 1 UCB: 122.37898078868042
Child 3 :
Number of children: 0
Value: 279.0 Number of trials: 1 UCB: 343.37898078868045
Child 4 :
Number of children: 0
Value: 16.0 Number of trials: 1 UCB: 80.37898078868042
Child 5 :
Number of children: 0
Value: -297.0 Number of trials: 1 UCB: -232.62101921131958
Child 6 :
Number of children: 0
Value: 76.0 Number of trials: 1 UCB: 140.37898078868042
Child 7 :
Number of children: 0
Value: 31.0 Number of trials: 1 UCB: 95.37898078868042
Child 8 :
Number of children: 0
Value: -121.0 Number of trials: 1 UCB: -56.62101921131958
Child 9 :
Number of children: 0
Value: 49

In [ ]:
def play_game(advesary_function, model : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        action = advesary_function(go_env)
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')

        if done:
            continue

        node = model.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node != None:
            action = node.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')
        turn_nr += 1
        if turn_nr > 300:
            break

    if node != None:
        model.back_propagation(node, go_env.reward())

def play_game_no_render(advesary_function, model : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        action = advesary_function(go_env)
        _, _, done, _ = go_env.step(action)

        if done:
            continue

        node = model.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node != None:
            action = node.action  
        _, _, done, _ = go_env.step(action)
        turn_nr += 1
        if turn_nr > 300:
            break

    if node != None:
        model.back_propagation(node, go_env.reward())
    
    return go_env

def play_model_vs_model(model1 : Monte_Carlo_Tree_Search, model2 : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        node1 = model1.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node1 != None:
            action = node1.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')

        if done:
            continue

        node2 = model2.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node2 != None:
            action = node2.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')
        turn_nr += 1
        if turn_nr > 300:
            break

    if node1 != None:
        model1.back_propagation(node1, go_env.reward())
    if node2 != None:
        model2.back_propagation(node2, go_env.reward())

def play_model_vs_model_no_render(model1 : Monte_Carlo_Tree_Search, model2 : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        node1 = model1.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node1 != None:
            action = node1.action  
        _, _, done, reward = go_env.step(action)

        if done:
            continue

        node2 = model2.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node2 != None:
            action = node2.action  
        _, _, done, reward = go_env.step(action)
        turn_nr += 1
        if turn_nr > 300:
            break
    return go_env

In [ ]:
for i in range(1,10):
    env = play_game_no_render(get_legal_move, model, copy.deepcopy(model.env))
    if env.done:
        print("Game finished within 300 turns:")
    else:
        print("Game stopped after 300 turns:")
        
    if env.reward() < 0:
        print("White won!")
    if env.reward() > 0:
        print("Black won!")
    if env.reward() == 0:
        print("It's a draw!")

Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─○─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Finding best move from: 25 available explored moves.
Game finished within 300 turns:
Black won!
Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─○
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Finding best move from: 0 available explored moves.
Game finished within 300 turns:
White won!
Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─○─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Finding best move from: 0 available explored moves.
Game finished within 300 turns:
Black won!
Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─○─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White

# Convolutional Neural Network

In [110]:
# Run on GPU if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device '{device}'")

Using device 'cuda:0'


In [118]:
class CNN(nn.Module):

    def __init__(self):
        super().__init__()
        # self.conv1 = nn.Conv2d(6, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, BOARD_SIZE**2 + 1)

    def forward(self, x):
        x = torch.from_numpy(x)
        x = x.to(device)
        x = x.float()
        # x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.softmax(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x    

In [119]:
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

def export_cnn(name="net"):
    torch.save(net.state_dict(), "cnns/" + name + ".pth")

def import_cnn(path):
    net = CNN()
    net.load_state_dict(torch.load(path))

def train_cnn(data):
    net.to(device)
    for epoch in range(2):  # loop over the dataset multiple times
        running_loss = 0.0
        
        for i in range(len(data)):
            inputs, labels = data[0][i], data[1][i]
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

        # for i, data in enumerate(trainloader, 0):
        #     # get the inputs; data is a list of [inputs, labels]
        #     inputs, labels = data

        #     # zero the parameter gradients
        #     optimizer.zero_grad()

        #     # forward + backward + optimize
        #     outputs = net(inputs)
        #     loss = criterion(outputs, labels)
        #     loss.backward()
        #     optimizer.step()

        #     # print statistics
        #     running_loss += loss.item()
        #     if i % 2000 == 1999:    # print every 2000 mini-batches
        #         print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        #         running_loss = 0.0


In [120]:
data = model.get_training_data_from_tree()
net = CNN() # reset neural network
train_cnn(data)

AttributeError: 'CNN' object has no attribute 'pool'